<h3>Goals</h3>
<ul>
    <li> Remove as much noise from Quest Drawing Text as possible. <b> Uses word Frequency </b></li>    
    <li> Find a way to correct spelling or something like that</li>   
    <li> Create a preliinary way of having users access the quest data</li> 
    <li> FIND ESTIMATED TIMINGS of searching our Quest Database</li> 
</ul>
    

<h2>Libraries + Settings + Functions</h2>

In [26]:
import pandas as pd
import numpy as np
import pyodbc
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import gensim
from gensim import corpora
from gensim import models
from gensim import similarities
def processText(text):
    #text_tokens = word_tokenize(text)
    text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [word.lower() for word in text_tokens]
    #text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    #text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    #text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    #text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens 
#might move below functions into above process text
def removeStops(text): #Remove stop words
    filtered = []
    for word in text:
        word = word.lower()
        #temp = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))),w) for w in correct_spellings if w[0]==word[0]]
        #sorted(temp, key = lambda val:val[0])[0][1]
        
        if word not in stop_words:
            filtered.append(word)
    return filtered
def stemming(text): # Remove stems from words
    words = []
    for word in text:
        words.append(stemmer.stem(word))
    return words

In [ ]:
query1 = """Select [Name], [DrawingTextAll]  from [Generation].[quest].[Drawings] WHERE DrawingTextAll IS NOT NULL"""
queryProperties ={"user": "", "password": "", "driver":"org.postgresql.Driver","spark.jars":"file:/home/postgresTest/postgresql-42.2.9.jar","spark.executor.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar","spark.driver.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar", 'partitionColumn':'TAGID_PI', 'lowerBound': '1100000000' , 'upperBound': '1510890005', 'numPartitions':'80', 'fetchsize':'100000'}
server = '' 
database = '' 
username = '' 
password = '
cnxn = pyodbc.connect('DRIVER=/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.10.so.1.1;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

#You may need to change the libmsodbcsql-17.8.s0.1.1 if we update it. Open a terminal cd out of /u01/ and start going down opt

p = pd.read_sql(query1,cnxn)
p['DrawingTextAll'][0]


<h2>Data Import </h2>
<ul>
    <li> Only work orders past 01/01/2020 </li>
    <li> Loading top 1000 Quest Items to not be super slow. Later need to change to test timings </li>
    </ul>

In [28]:
host = r''
dbSQL = ''
user = ''
pwd = ''
#Work Orders

woSelect= f"""SELECT [WONUM]
      ,[LOCATION]
      ,[DESCRIPTION]
  FROM [Generation].[agent].[MaximoWorkOrders]
  Where cast(STATUSDATE as date) > '01/01/2020'
""" 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+host+';DATABASE='+dbSQL+';UID='+user+';PWD='+ pwd)
WOdata = pd.read_sql(woSelect, cnxn)
WOdata = WOdata.drop(columns=['WONUM'])

WOdata['Edited1'] = [list() for x in range(len(WOdata.index))]
WOdata['Edited1'] = WOdata['DESCRIPTION'].apply(processText)


#Quest
query1 = """Select TOP(1000) [Name], [DrawingTextAll]  from [Generation].[quest].[Drawings] WHERE DrawingTextAll IS NOT NULL"""
queryProperties ={"user": "GENENG", "password": "postgres", "driver":"org.postgresql.Driver","spark.jars":"file:/home/postgresTest/postgresql-42.2.9.jar","spark.executor.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar","spark.driver.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar", 'partitionColumn':'TAGID_PI', 'lowerBound': '1100000000' , 'upperBound': '1510890005', 'numPartitions':'80', 'fetchsize':'100000'}
server = '' 
database = '' 
username = '' 
password = ''
cnxn = pyodbc.connect('DRIVER=/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.10.so.1.1;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#You may need to change the libmsodbcsql-17.8.s0.1.1 if we update it. Open a terminal cd out of /u01/ and start going down opt
global questData
questData = pd.read_sql(query1,cnxn)
questData['Edited1'] = [list() for x in range(len(questData.index))]
questData['Edited1'] = questData['DrawingTextAll'].apply(processText)

cnxn.close()


#questData.head()

/home/spark/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/spark/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<h2>Getting Word Frequency from WO</h2>
<ul> <li> Running a spellchecker on our words as well so we get correct spellings before we add to dictionary </li>
    <li> We can afford to use a more verbose spellchecker since we have lower amount of words</li>
    
</ul>



In [77]:
from nltk import FreqDist
from spellchecker import SpellChecker


spell = SpellChecker(distance = 2)
#spell.word_frequency.load_dictionary("WOWordFreq.json")


print(WOdata.columns)
#Aggregating Columns to one list
woText = WOdata.groupby(lambda _ : True)['DESCRIPTION'].transform(lambda x : ' '.join(x)).iat[0]

print('tokenize')
woText = nltk.tokenize.RegexpTokenizer('[A-Z]\w+').tokenize(woText)

print('stops + stemming')
woText = removeStops(woText)
# stemmign takes a hella long time bro
#woText = stemming(woText)


#print('\n\n frequency')
woText = FreqDist(woText)
export = pd.DataFrame(list(woText.items()), columns = ["Word","Frequency"])

#spell correction
print('spelling')
print('words i\'m correcting',len(export['Word']))
#We spell correct after aggregating to freq dist to avoid repeat and saving time from 160k wrods to 18k
#and we reaggregate with summnation
#export['Word'] = export['Word'].apply(spell.correction)
#export = export.groupby(['Word']).sum()
print('done')

Index(['LOCATION', 'DESCRIPTION', 'Edited1', 'Score'], dtype='object')
tokenize
stops + stemming
spelling
words i'm correcting 17989
done


In [ ]:
#export = pd.read_csv("SCWOWordFreq.txt", sep=' ', header = None)
#export = pd.read_csv("words/SCWOWordFreq.txt", sep=' ', header = None)

export['Word']= export[0]
export['Frequency'] = export.astype({1: 'int64'})[1]
export = export[['Word', 'Frequency']]

In [83]:
#export.hist(column=['Frequency'], range= (10,40))
#Cutting depending on frequency
print('cutting')

WOWordFreq = export
WOWordFreq = WOWordFreq[WOWordFreq['Frequency'] < 20000]
WOWordFreq = WOWordFreq[WOWordFreq['Frequency'] > 200]

WOWordFreq = WOWordFreq.sort_values("Frequency", ascending = False)
WOWordFreq = WOWordFreq.reset_index()
'''
print('saving export sincei  don\'t want to re process')
with open('SCWOWordFreq.txt', 'w') as f:
    #temp = export.reset_index() #use if you get it from FreqDist,
    temp = export
    for w in range(len(temp)):
       f.write(str(temp['Word'][w]) + " " + str(temp['Frequency'][w]) + '\n')'''

WOWordFreq

cutting


,index,Word,Frequency
0,258,inspections,18177
1,99,oil,16516
2,59,pump,15796
3,111,air,15602
4,908,checks,14342
...,...,...,...
987,5026,guards,202
988,192,term,202
989,128,properly,201
990,4441,amps,201


In [81]:
export.sort_values("Frequency", ascending = False).head(20)
#export.hist(column=['Frequency'], range= (100,1000))

,Word,Frequency
180,pm,149192
1877,weekly,39019
21,unit,33776
1590,daily,24731
907,monthly,20743
258,inspections,18177
99,oil,16516
59,pump,15796
111,air,15602
908,checks,14342


<h2> Setting up SymSpell for Spell Checker </h2>
<ol> <li> Vocab comes from either A. WO Word Frequency or B. Library's prebuilt, or C. Combined </li>
    <li> SymSpell is the fastest, but performs the least amount of transformations to find words </li>
    <li> Consider condensing words, ie turning aaaaaa to aa </li>
    <li> Looking to remove nonwords post spellchecked </li>
    <li> Some test code to work on a singular drawing </li>  
    </ol>

In [86]:
import pkg_resources
from symspellpy import SymSpell, Verbosity
import json
sym_spell = SymSpell()

#Default 
#sym_spell.load_dictionary("SymSpellDict_en_82_765.txt", 0, 1)
'''
with open('WOWordFreq.txt', 'w') as f:
    for w in range(len(WOWordFreq)):
        f.write(str(WOWordFreq['Word'][w]) + " " + str(WOWordFreq['Frequency'][w]) + '\n')
    '''
corpus_path = "words/CutWOWordFreq.txt"
sym_spell.create_dictionary(corpus_path)
#print(sym_spell.words)

#Wait this is uneceessary - Alvin 9/15/2022

True

In [58]:
#To make, copy SymSpellDict_en_82_765.txt as CombinedFreq.txt and run below code
#need to remake it
'''
#Careful you need to fix it in the middle and write 1 extra newline at bottom
with open("CombinedFreq.txt", 'a') as f:
    for w in range(len(WOWordFreq)):
        f.write(str(WOWordFreq['Word'][w] + " " + str(WOWordFreq['Frequency'][w]) + '\n'))
'''

In [ ]:
from symspellpy import SymSpell, Verbosity
from nltk.corpus import words
sym_spell = SymSpell()
#sym_spell.load_dictionary("SymSpellDict_en_82_765.txt", 0, 1)
sym_spell.load_dictionary("words/cutWOWordFreq.txt", 0, 1)
#sym_spell.load_dictionary("CombinedFreq.txt", 0, 1) #takes a while sicne it has to iterate through more
questData['Edited2'] = questData['Edited1'].apply(' '.join)
probe = 1
spellingTestList = questData['Edited2'][probe]
def spellChecker(text):
    text = text.split(' ')
    final = ''
    for w in range(len(text)):
        if len(text[w]) != 1: #skipping single letters pray it doesn't matter.
            suggestions = sym_spell.lookup(text[w].lower(), Verbosity.CLOSEST, max_edit_distance=2, ignore_token = None,   include_unknown=True)
            for suggestion in suggestions:
                #I would love to check for words that are actualyl english but the time it takes is rather.. long
                #if str(suggestion).split(',')[0] in words.words()
                final = final + str(suggestion).split(',')[0] + ' '
                break
    return final

#single testing
spellingTestList = spellChecker(spellingTestList)
print(questData['Name'][probe])
print(questData['Edited2'][probe])
print(spellingTestList)

In [90]:
'gravimetric' in export['Word']

False

In [55]:
questData['spellChecked'] = questData['Edited2'].apply(spellChecker)

In [ ]:
questData['spellChecked'][3]

In [54]:
for suggestion in sym_spell.lookup('custom'.lower(), Verbosity.CLOSEST, max_edit_distance=2,  include_unknown=True):
    print(suggestion)

custom, 3, 0


In [ ]:
WOWordFreq['Frequency'][WOWordFreq['Word'].to_list().index('channel')]


<h2>Pyspellchecker</h2>
<ul><li>Gotta say pyspellchecker is pretty slow, but probably best bet if you want good quality?
    </li></ul>


In [76]:
from spellchecker import SpellChecker

#I can't believe pandas json formatting sucks and doesn't work and i'm too lazy to work around it.
with open('WOWordFreq.json', 'w') as f:
    f.write("{")
    for w in range(len(WOWordFreq)):
        f.write(f"\"{WOWordFreq['Word'][w]}\":{str(WOWordFreq['Frequency'][w])}")
        if w != len(WOWordFreq) - 1:
            f.write(',')
    f.write("}")

spell = SpellChecker(distance = 2)
#spell.word_frequency.load_dictionary("WOWordFreq.json")
spell.correction('fgd')

NameError: name 'WOWordFreq' is not defined

In [ ]:
def spellChecker(text):
    words = text.split(' ')
    final = ''
    for w in range(len(words)):
        suggestions =spell.correction(words[w].lower())
        #if suggestions in WOWordFreq['Word'].to_list():
            #final = final + suggestions + ' '
        final = final + suggestions + ' '
        
    return final
spellingTestList = questData['DrawingTextAll'][probe]
spellingTestList = spellChecker(spellingTestList)
print(questData['Name'][probe])
print(spellingTestList)

In [ ]:
spellChecked = questData['DrawingTextAll'].apply(spellChecker)

In [ ]:
spellChecked[30]

<h1>Word2Vec Model using WO Data</h1>

In [ ]:
WOdata['Edited1'] = [list() for x in range(len(WOdata.index))]
WOdata['Edited1'] = WOdata['DESCRIPTION'].apply(processText)
WOdata.head()

In [8]:
texts = WOdata['Edited1'].to_list()
'''
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]
'''
dictionary = corpora.Dictionary(WOdata['Edited1'])
corpus = [dictionary.doc2bow(text) for text in WOdata['Edited1']]
#lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=16)

In [63]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

model = gensim.models.Word2Vec(sentences=WOdata['Edited1'].to_list())
#min_count
#ns_exponent = 0.75 #negative means low-frequency higher than high-fre
#trim_rule = lambda x: 
#sample = threshhold for which-h
wv = model.wv
#vec_king = model.wv['Coal']
for index, word in enumerate(wv.index_to_key):
    if index ==10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/8302 is pm
word #1/8302 is 1
word #2/8302 is 2
word #3/8302 is weekly
word #4/8302 is unit
word #5/8302 is 3
word #6/8302 is 4
word #7/8302 is daily
word #8/8302 is monthly
word #9/8302 is oil


In [36]:
model.save('WOWord2Vec')


In [3]:
model = gensim.models.Word2Vec.load('WOWord2Vec')

In [65]:
target = 'broke'
model.wv.most_similar(target)

[('broken', 0.8694186210632324),
 ('missing', 0.8669424057006836),
 ('bent', 0.8198492527008057),
 ('loose', 0.8186651468276978),
 ('fell', 0.7903898358345032),
 ('stripped', 0.7291839718818665),
 ('sheared', 0.7282971143722534),
 ('hitting', 0.7082007527351379),
 ('worn', 0.7074403166770935),
 ('arm', 0.7035567164421082)]

<h1>Using The WO Word2Vec Model for Similarity in Quest </h1>

In [74]:
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#Need to clean edited1 since i think the noise is hurting the data comparison
sentence_1 = processText("coal feed rate indicators for")
print(sentence_1)
def attempt(target):
    return model.wv.wmdistance(sentence_1, target)
#questData['Score'] = questData['Edited1'].apply(attempt)
questData['Score'] = questData['spellChecked'].apply(attempt)
questData = questData.sort_values("Score", ascending = False)


['coal', 'feed', 'rate', 'indicators']


In [ ]:
questData[['Name','DrawingTextAll','spellChecked']].values[0]

In [ ]:
questData.head(10)

In [223]:
WOdata['Score'] = WOdata['Edited1'].apply(attempt)
WOdata.sort_values("Score", ascending = True).head()

,LOCATION,DESCRIPTION,Edited1,Edited2,Score
10478,MC000CHGVFDJA-MTR,JA Feeder Bolt Loose,"[ja, feeder, bolt, loose]",[],0.630830
198072,MC000CHGVFDE2,E2 feeder has a spring bolt loose.,"[e2, feeder, spring, bolt, loose]",[],0.731587
167091,MC000PLTXXX,"Door Hinge falling apart, screws falling out h...","[door, hinge, falling, apart, screws, falling,...",[],0.750803
242,TC000PRDXXX,Install loose tray cover,"[install, loose, tray, cover]",[],0.825695
153479,TC001ESPRAPXXX,B32-2 Has a Loose Cylinder Clamp might be broke,"[b32, 2, has, loose, cylinder, clamp, might, b...",[],0.829852
...,...,...,...,...,...
155359,MC000CHGSRC---BOOMCONVY,cancelled redundent,"[cancelled, redundent]",[],inf
46576,GHCY-S-GLITALLS-GSGLT,VOID,[void],[],inf
195331,MC000STBXXX,\,[],[],inf
124899,GHAP-CCRGYP01AFFDPMP,HHTRRH,[hhtrrh],[],inf


In [ ]:
#quest probing
probe = 3
print(questData['Name'][probe])
print(questData['DrawingTextAll'][probe])
print(questData['spellChecked'][probe])


In [145]:
'ee' in words.words()

False

In [ ]:
from nltk.corpus import words
'ee' in words.words()
final = []
for w in questData['spellChecked'][probe].split(' '):
    if w in words.words():
        final.append(w)
final

<h1> Testing Nmslib Indexer </h1>
<a href = 'https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/nmslibtutorial.ipynb'>Lonk</a>

In [38]:
from gensim.similarities.nmslib import NmslibIndexer
nmslib_index = NmslibIndexer(model, {'M': 100, 'indexThreadQty': 1, 'efConstruction': 100}, {'efSearch': 10})
#M and efConstruction is accuracy, but onger indexing
#indexThreatQty is nubmer of theready
#EfSearch higher is better but longer

In [56]:
model = gensim.models.Word2Vec.load('WOWord2Vec')
#model = gensim.models.Doc2Vec.load('doc2vecWOData')

In [57]:
vector = model.wv["absorber"]

In [58]:
approximate_neighbors = model.wv.most_similar([vector], topn=11, indexer=nmslib_index)

In [59]:
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

Approximate Neighbors
('absorber', 1.0)
('abs', 0.7976338863372803)
('wfgd', 0.5707395076751709)
('fgd', 0.5479655861854553)
('agitator', 0.4636227488517761)
('agitators', 0.4607580304145813)
('arp', 0.436490535736084)
('hydrocyclone', 0.4069739580154419)
('recycle', 0.4054768681526184)
('demineralizer', 0.3975650668144226)
('lst', 0.39382314682006836)


In [42]:
normal_neighbors = model.wv.most_similar([vector], topn=11)
print("\nNormal (not nmslib-indexed) Neighbors")
for neighbor in normal_neighbors:
    print(neighbor)


Normal (not nmslib-indexed) Neighbors
('fgd', 1.0)
('abs', 0.6093549728393555)
('absorber', 0.5479655265808105)
('wfgd', 0.4383356273174286)
('bromine', 0.4138839542865753)
('eliminator', 0.40072569251060486)
('hydrocyclone', 0.37773463129997253)
('cw', 0.3742797374725342)
('scrubber', 0.3714107275009155)
('lpsw', 0.36952903866767883)
('pier', 0.3678264021873474)


In [ ]:
WOsearch = WOdata
sentence_1 = processText("fgd")
print(sentence_1)
def attempt(target):
    return model.wv.wmdistance(sentence_1, target, indexer=nmslib_index)
#questData['Score'] = questData['Edited1'].apply(attempt)
WOsearch['Score'] = WOsearch['Edited1'].apply(attempt)
WOsearch = WOsearch.sort_values("Score", ascending = False)


In [ ]:
WOsearch['Score'] = WOsearch['Edited1'].apply(attempt)
WOsearch.sort_values("Score", ascending = True).head(5)

<h1> Word Filter? </h2>

In [97]:
acronyms = []
with open('words/acronyms.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        acronyms.append(line.split('\n')[0])

In [ ]:
woWords = []
with open('words/CutWOWordFreq.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        woWords.append(line.split(' ')[0])
#woWords

In [75]:
len(acronyms)

1572

In [121]:
len(stop_words.union(set(acronyms + woWords + words.words())))

237481

In [113]:
len(set(acronyms + words.words()).union(stop_words))


237315

In [37]:
combinedFreq = pd.read_csv("words/FullCombinedFreq.txt", sep=' ', header = None)
combinedFreq.sort_values(1, ascending = False)
combindedFreq[0].values[102520]

In [84]:
symSpellDict = pd.read_csv("words/SymSpellDict_en_82_765.txt", sep=' ', header = None)
WOWordFreq = pd.read_csv("words/SCWOWordFreq.txt", sep=' ', header = None)

In [ ]:
import datetime
print(datetime.datetime.now())
questData['Filetered'] = questData['Edited1'].apply(lambda x : [w for w in x if w in set(acronyms + words.words())])
print(datetime.datetime.now())

2022-09-16 11:30:53.837951


In [85]:
WOWordFreq[WOWordFreq[0] == 'ee']

,0,1
3649,ee,2


In [89]:
len(combinedFreq[0].unique())

93243

In [132]:
import random

In [163]:
'dimension' in words.words()

True

In [ ]:
p = questData.iloc[round(random.random()*1000)]
print(p['DrawingTextAll'])
p['Name']
#3231905
#3129769

In [ ]:
from nltk.corpus import words
#p = questData['DrawingTextAll'][0]
split = p['DrawingTextAll'].split(' ')
def filter(word):
    if word in set(acronyms + words.words()) or word.isupper():
        return word
' '.join([w for w in split if w.lower() in (set(acronyms + words.words()) or w.isupper()) and len(w) > 2 ])